# 악기 브랜드(Gibson) 사이트 크롤링 과제
### 1. 목표: 깁슨 사이트에서 제공하는 모든 악기의 데이터를 수집한다. 
### 2. 자료 수집: 이름, 하이퍼링크, 가격, 이미지, 상세스펙
### 3. CSV파일로 저장

In [1]:
# 사용할 모듈
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import os
import pandas as pd
        
# 모듈 명령어
# dir(Gibson_Crawling) 으로 호출
intro = """Les-Paul, SG, ES, Acoustic, Designer, Bass 를 'crl.run( )'안에 입력하세요. 입력한 값에 대한 깁슨 홈페이지에 있는 모든 깁슨 모델을 제공합니다."""
maker = 'jungho kang'

BASE_DIR = os.getcwd() # os명령어. get current work directory 현재 작업 주소

class crl():
    # def __init__(self, base_dir = BASE_DIR):
    def __init__(self, base_dir):
        self.base_dir = BASE_DIR
    
    def run(model):
                 
        PICTURE_DIR = os.path.join(BASE_DIR, 'guitar_pictures_{}'.format(model))
        # Ex. 폴더명: guitar_picture_Acoustic
        if not os.path.exists(PICTURE_DIR):
            os.makedirs(PICTURE_DIR) # 만약 경로가 없으면, 생성하겠습니다.
    
        guitar_list = [] # 새로운 빈 리스트를 생성합니다.
        gibson_usl = "https://www.gibson.com/Guitars/" + model # 파라미터 보다 쉽다.
# params = {
#     'Guitars': 'Guitars',
#     'Les-Paul': 'Les-Paul'
# }
# gibson_usl
        resp = requests.get(gibson_usl)
        # resp
        # resp.url
        # resp.content
        soup = BeautifulSoup(resp.content, 'html.parser')
        # soup
        gibson_contents = soup.find('div', class_='row pb-5').find_all('div', class_='col-sm-12 col-md-6 col-lg-4 px-2 mb-4')
        # gibson_contents

        for div in gibson_contents:
            guitar_dict = {}

            a_tag = div.find('h4').find('a')
    # print(a_tag)
            guitar_dict['link'] = 'https://www.gibson.com'+a_tag['href']
            guitar_dict['name'] = a_tag.text
    # print(a_tag['href'])
    
            b_tag = div.find('div', class_='price-label ml-3').find('span')
            guitar_dict['price'] = b_tag.text
    # print(b_tag)
            
            # 상세페이지 내용 가져오기 
            detail_resp = requests.get('https://www.gibson.com'+a_tag['href'])
            # detail_resp
            # detail_resp.url
            # detail_resp.content
            detail_soup = BeautifulSoup(detail_resp.content, 'html.parser')
            # detail_soup
            detail_content = detail_soup.find('div', class_='gibson-card py-4')
            # detail_content

            d_tag = detail_content.find('div', class_='spec-section body-specs pb-4 mb-4').text.split('\n')
            e_tag = detail_content.find('div', class_='spec-section neck-specs mb-4').text.split('\n')
            f_tag = detail_content.find('div', class_='spec-section hardware-specs mb-4').text.split('\n')
            g_tag = detail_content.find('div', class_='spec-section electronics-specs mb-4').text.split('\n')
            h_tag = detail_content.find('div', class_='spec-section miscellaneous-specs mb-4').text.split('\n')

            guitar_dict['Body'] = d_tag
            guitar_dict['Neck'] = e_tag
            guitar_dict['Hardware'] = f_tag
            guitar_dict['Electronics'] = g_tag
            guitar_dict['Miscellaneous'] = h_tag
    
            guitar_list.append(guitar_dict)
    
            c_tag = div.find('img')
    # print(c_tag)
    # print(c_tag['src'])
    
            img_url = c_tag['src']
            img_resp = requests.get(img_url)
    # img_url
    # imgresp.content
            p_filename = img_url.split('/')[-4:]
            filename = "".join(p_filename)
    # filename
    
            guitar_picture_path = os.path.join(PICTURE_DIR, filename)  
            with open(guitar_picture_path, 'wb') as f:
                f.write(img_resp.content)
    
    # 나중에 확인해 볼 코드 
    # exec("{0} = crl.run('{0}')".format('Acoustic'))
    # Acoustic
    
        
        return guitar_list

# CSV 파일 쓰기

In [2]:
Bass = crl.run("Bass")
Bass

[{'link': 'https://www.gibson.com/Guitar/USANRD503/EB-Bass-4-String-2019',
  'name': 'EB Bass 4 String 2019',
  'price': ' $1,149.00',
  'Body': ['',
   'Body',
   '',
   'Body Material',
   'Swamp Ash',
   '',
   '',
   'Weight Relief',
   'None',
   '',
   '',
   'Finish',
   'Satin Nitrocellulose Lacquer',
   '',
   ''],
  'Neck': ['',
   'Neck',
   '',
   'Neck Material',
   'Maple',
   '',
   '',
   'Neck Profile',
   'Rounded',
   '',
   '',
   'Fingerboard Material',
   'Rosewood',
   '',
   '',
   'Fingerboard Radius',
   '12" / 304.8mm',
   '',
   '',
   'Number of Frets',
   '24',
   '',
   '',
   'Frets',
   'Medium - Cryogenically Treated',
   '',
   '',
   'Nut Material',
   'Tektoid',
   '',
   '',
   'Nut Width',
   '1.625" / 41.28mm',
   '',
   '',
   'End-of-Board Width',
   '2.484" / 63.09mm',
   '',
   '',
   'Inlays',
   'Acrylic Dots',
   '',
   ''],
  'Hardware': ['',
   'Hardware',
   '',
   'Finish',
   'Chrome',
   '',
   '',
   'Bridge',
   'BABICZ Full Contac

In [3]:
guitar_excel = Bass
df = pd.DataFrame(guitar_excel)
# df

In [4]:
df.to_csv('gibsun_Bass.csv')